In [1]:
from constraint import Problem
import traceback
import copy


from utils import get_solutions_for_one_variable, get_other_variables, apply_constraint

In [2]:
DEBUG = False

In [3]:
class Histories:
    def __init__(self):
        self.content = []
        self.step = 1

    def add(self, var1, var2, operation, result):
        self.content.append(
            {"var1": var1, "var2": var2, "operation": operation, "result": result}
        )

        self.step += 1

    def has_already_used(self, var1, var2, operation):
        for history in self.content:
            operation_match = operation == history["operation"]
            vars_match = (var1 == history["var1"] and var2 == history["var2"]) or (
                var2 == history["var1"] and var1 == history["var2"]
            )

            if operation_match and vars_match:
                return True
        return False

    def compact(self):
        return list(
            map(
                lambda h: f"{h['var1']} {h['var2']} {h['operation']} {h['result']}",
                self.content,
            )
        )


In [ ]:
def check_constraint_validy(problem, var1, var2, operation, result):
    new_problem = copy.deepcopy(problem)

    apply_constraint(new_problem, var1, var2, operation, result)

    for people in get_other_variables(new_problem):
        candidates = get_solutions_for_one_variable(new_problem, people)
        if len(candidates) == 0:
            yes_no = input(f"No candidates found for {people} if using this cmd, continue with this ? y/N")
            if yes_no == "y":
                continue
            else:
                raise Exception(f"No candidates found for {people} if using this cmd")
        if len(candidates) == 1:
            print(f"One candidate left for {people} == {candidates}")


def add_command(problem, cmd: str):
    # cmd sous la forme "VAR1 VAR2 OPERATION RESULT"
    parsed_cmd = cmd.upper().split()

    var1 = parsed_cmd[0]
    var2 = parsed_cmd[1]
    operation = parsed_cmd[2]
    result = int(parsed_cmd[3])

    check_constraint_validy(problem, var1, var2, operation, result)
    apply_constraint(problem, var1, var2, operation, result)

    return var1, var2, operation, result

def get_best_operation_between_two_variable(problem: Problem, var1: str, var2: str, histories: Histories):
    operation = ""
    
    # PRIORITY 0: to set boundaries if any known boundaries
    if histories.step == 1:
        operation = "ADD"
        return operation + " (First step)"
    
    candidates = get_solutions_for_one_variable(problem, var1)

    # PRIORITY 0: use add and be lucky enough to get direct addition
    if len(candidates) == 1:
        operation = "ADD"
        if not histories.has_already_used(var1, var2, operation):
            return operation + " (Lucky Test)"

    # PRIORITY 1: if candidates have differ by last digits
    last_digits = set(c % 10 for c in candidates)
    if len(last_digits) > 1:
        # Usefull if candidates have to many units number
        operation = "MUL"
        if not histories.has_already_used(var1, var2, operation):
            return operation + " (Last digits)"
    
    # PRIORITY 2: if candidates differ by tens digit
    tens_digits = set(c // 10 for c in candidates)
    if len(tens_digits) > 1:
        operation = "ZER"
        if not histories.has_already_used(var1, var2, operation):
            return operation + " (Tens digits)"
        
    # PRIORITY 3: when spread is large
    spread = max(candidates) - min(candidates)
    if spread > 20:
        operation = "DIV"
        if not histories.has_already_used(var1, var2, operation):
            return operation + " (Large spread)"
    
    # DEFAULT: to multiplication for small difference
    return "MUL (Default)"

def get_best_operation_to_know_my_variable(problem: Problem, me: str, histories: Histories):

    operations = []
    other_peoples = get_other_variables(problem, me)
    for people in other_peoples:
        operations.append(
            {
                "var": people,
                "operation": get_best_operation_between_two_variable(problem, me, people, histories)
            }
        )

    return operations


In [5]:
MAX_NUMBER = 100
MIN_NUMBER = 1

me = "K"
peoples = [me, "L", "P", "O"]
histories = Histories()

problem = Problem()
problem.addVariables(peoples, range(MIN_NUMBER, MAX_NUMBER + 1))


In [22]:
# voir les résultat possible pour une varible
# changer la valeur de var

# take times if launched without any constraint
solutions = "par défaut"

if histories.step == 1:
    result = input("Il s'agit du première étape, afficher les solutions ? (Y/n)")
    if result == "y":
        solutions = []
        for p in problem._variables:
            solutions.append({
                "var": p,
                "candidates": get_solutions_for_one_variable(problem, p)
            })
else:
    solutions = []
    for p in problem._variables:
        solutions.append({
            "var": p,
            "candidates": get_solutions_for_one_variable(problem, p)
        })
    
solutions

[{'var': 'K', 'candidates': [97, 77, 81, 87, 91]},
 {'var': 'L', 'candidates': [96, 76, 82, 86, 92]},
 {'var': 'P', 'candidates': [100, 70, 80, 90, 60]},
 {'var': 'O',
  'candidates': [1,
   3,
   11,
   13,
   21,
   23,
   31,
   33,
   41,
   43,
   51,
   53,
   61,
   63,
   71,
   73,
   81,
   83,
   91,
   93]}]

In [28]:
get_best_operation_to_know_my_variable(problem, me, histories)

[{'var': 'L', 'operation': 'MUL (Default)'},
 {'var': 'P', 'operation': 'ADD (Lucky Test)'},
 {'var': 'O', 'operation': 'ADD (Lucky Test)'}]

In [29]:
# ecrire le commande sous la forme : 
cmd = input("> VAR1 VAR2 OPERATION RESULT")

if cmd:
    try:
        var1, var2, operation, result = add_command(problem, cmd)
        histories.add(var1, var2, operation, result)
        print("nouvelle contrainte ajouté!")
    except Exception as e:
        print("erreur (vérifier vos inputs)", e)
        if DEBUG:
            traceback.print_exc()
else:
    print("cmd non valide, vérifier les entrées")

# Show histories of applied constraint
histories.compact()

One candidate left for K == [87]
One candidate left for L == [86]
One candidate left for P == [90]
One candidate left for O == [83]
nouvelle contrainte ajouté!


['K L ADD 173',
 'K L MUL 2',
 'K P MUL 0',
 'K P ZER 1',
 'K O MUL 1',
 'K O ZER 0',
 'K L ZER 0',
 'K P ADD 177']